In [35]:
%matplotlib inline  
import math
import matplotlib.pyplot as plt
import numpy as np
import fiona
import pygeodesy
from shapely.geometry import shape
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [36]:

# 计算方位角
def get_bearing(pnt0, pnt1):
    lng0, lat0 = pnt0
    lng1, lat1 = pnt1
    return pygeodesy.compassAngle(lat0, lng0, lat1, lng1)

# 计算距离
def get_length(pnt0, pnt1):
    lng0, lat0 = pnt0
    lng1, lat1 = pnt1
    return pygeodesy.haversine(lat0, lng0, lat1, lng1)
    

In [54]:
def get_frequency(shp_path='./road/guanzhou.shp', bins=60):
    y = np.zeros(bins, dtype=np.float64)
    width = 360 / bins
    
    c = fiona.open(shp_path)
    for fea in c:
        geom = shape(fea['geometry'])
        try:
            coords = geom.coords
        except:
            continue
        pre_pnt = coords[0]
        for pnt in coords[1:]:

            bearing1 = get_bearing(pre_pnt, pnt)
            bearing2 = get_bearing(pnt, pre_pnt)
            length = get_length(pre_pnt, pnt)

            k1 = int(bearing1 // width)
            k2 = int(bearing2 // width)
            y[k1] += length
            y[k2] += length
            pre_pnt = pnt

    c.close()
    return y / y.sum()

In [76]:
def plot_historgram(ax, frequency, bins, title):
    

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')
    
    x = np.array([ang * 2 * np.pi / bins for ang in range(0, bins)])
    width = 2 * np.pi / bins



    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                      color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)

    ax.set_ylim(top=frequency.max())


    title_font = {'family':'sans-serif', 'size':20, 'weight':'bold'}
    ax.set_title(title, y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ytick_font = {'family':'sans-serif', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    xtick_font = {'family':'sans-serif', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    
    ax.tick_params(axis='x', which='major', pad=-2) 
    

In [79]:
places = {
    
    '深圳': './road/shenzhen.shp',
    '广州': './road/guanzhou.shp',
    '珠海': './road/zhuhai.shp',
    '韶关': './road/shaoguan.shp',
    '汕头': './road/shantou.shp',
    '佛山': './road/foshan.shp',
    '江门': './road/jianmen.shp',
    '湛江': './road/zhanjian.shp',
    '茂名': './road/maoming.shp',
    '肇庆': './road/zhaoqing.shp',
    '惠州': './road/huizhou.shp',
    '梅州': './road/meizhou.shp',
    '汕尾': './road/shanwei.shp',
    '河源': './road/heyuan.shp',
    '阳江': './road/yanjian.shp',
    '清远': './road/qinyuang.shp',
    '东莞': './road/donguan.shp',
    '中山': './road/zhongshang.shp',
    '潮州': './road/chaozhou.shp',
    '揭阳': './road/jieyan.shp',
    '云浮': './road/yunfu.shp',    
}
bins = 60


place_frequency = {}

for place_name, place_path in places.items():
    frequency = get_frequency(place_path, bins)
    place_frequency[place_name] = frequency


In [81]:
place_frequency['珠海']

array([0.01460315, 0.01079035, 0.01034135, 0.01312549, 0.01177365,
       0.01426584, 0.01359801, 0.01294694, 0.01905305, 0.01810405,
       0.02297699, 0.02541798, 0.02497374, 0.02642604, 0.01971873,
       0.01773987, 0.01169558, 0.01004228, 0.01136518, 0.01069611,
       0.01172762, 0.01653228, 0.01618309, 0.01991618, 0.02043123,
       0.01791623, 0.01939256, 0.0179404 , 0.022772  , 0.01753406,
       0.01460315, 0.01079035, 0.01034135, 0.01312549, 0.01177365,
       0.01426584, 0.01359801, 0.01294694, 0.01905305, 0.01810405,
       0.02297699, 0.02541798, 0.02497374, 0.02642604, 0.01971873,
       0.01773987, 0.01169558, 0.01004228, 0.01136518, 0.01069611,
       0.01172762, 0.01653228, 0.01618309, 0.01991618, 0.02043123,
       0.01791623, 0.01939256, 0.0179404 , 0.022772  , 0.01753406])

In [84]:

n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'}, squeeze=False)
axes = [item for sublist in axes for item in sublist]

for ax, place_name in zip(axes, place_frequency.keys()):
    
    frequency = place_frequency[place_name]
    plot_historgram(ax, frequency, bins, place_name)
    

# title的锚点是 （middle, top）, 默认x=0.5, y=0.98
suptitle_font = {'family':'sans-serif', 'fontsize':25, 'fontweight':'normal', 'y':1.07}
fig.suptitle('城市街道网络方向', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
plt.gcf().savefig('./street-orientations.png', dpi=360, bbox_inches='tight')
plt.close()
